In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/nlp-2021-vda/annotate_data/
!pwd

If you want to compute statistics on our annotated data, download the data from google cloud.

In [ ]:
import os
if not os.path.isdir('./datasets'):
  os.mkdir('./datasets')
!gsutil -m cp -r gs://nlp-lab/annotate_data/datasets/statistics_data ./datasets/

In [ ]:
import json
import numpy as np
from sklearn.metrics import cohen_kappa_score

In [ ]:
with open('datasets/statistics_data/ananta.json') as f:
  ananta = json.load(f)

with open('datasets/statistics_data/vivi.json') as f:
  vivi = json.load(f)

with open('datasets/statistics_data/sophie.json') as f:
  sophie = json.load(f)

In [ ]:
# counter represents how many same (query, response) pairs each annotated 
def store_answers(ananta, vivi, sophie, counter):
  ananta_ans = []
  vivi_ans = []
  sophie_ans = []
  for i in range(counter):
    answer1 = ananta['ananta'][i]['best']
    ananta_ans.append(answer1)
    answer2 = vivi['vivi'][i]['best']
    vivi_ans.append(answer2)
    answer3 = sophie['sophie'][i]['best']
    sophie_ans.append(answer3)
  return ananta_ans, vivi_ans, sophie_ans

In [ ]:
def calculate_confusion_matrix(ananta_ans, vivi_ans, sophie_ans):
  m = np.zeros((3, 3))
  all_agreement = 0
  for i in range(len(ananta_ans)):
      if ananta_ans[i] == vivi_ans[i] and vivi_ans[i]== sophie_ans[i]:
        all_agreement += 1
        m[0, 1] += 1
        m[0, 2] += 1
        m[1, 0] += 1
        m[1, 2] += 1
        m[2, 0] += 1
        m[2, 1] += 1
      
      elif ananta_ans[i] == vivi_ans[i]:
        m[0, 1] += 1
        m[1, 0] += 1

      elif ananta_ans[i] == sophie_ans[i]:
        m[0, 2] += 1
        m[2, 0] += 1

      elif sophie_ans[i] == vivi_ans[i]:
        m[1, 2] += 1
        m[2, 1] += 1
    
  return m, all_agreement

In [ ]:
counter = 100
ananta_ans, vivi_ans, sophie_ans = store_answers(ananta, vivi, sophie, counter)
print("Ananta and Vivi: ", cohen_kappa_score(ananta_ans, vivi_ans))
print("Ananta and Sophie: ", cohen_kappa_score(ananta_ans, sophie_ans))
print("Vivi and Sophie: ", cohen_kappa_score(vivi_ans, sophie_ans))

In [ ]:
m, all_agreement = calculate_confusion_matrix(ananta_ans, vivi_ans, sophie_ans)

In [ ]:
print("All agreement: ", all_agreement/counter * 100, "%")
print("Ananta and Vivi: ", m[0, 1]/counter * 100, "%")
print("Ananta and Sophie: ", m[0, 2]/counter*100, "%")
print("Vivi and Sophie: ", m[1, 2] / counter * 100, "%")